<a href="https://colab.research.google.com/github/jw-jang/BME3890/blob/main/1_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install mrjob
!pip install mrjob

In [ ]:
from google.colab import files

# upload dataset
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
from google.colab import files

# upload mrjob.conf under current directory(/content/mrjob.conf)
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
%%file 1_count.py
from mrjob.job import MRJob
import json

'''
Count the number of tweets.
Parse tweets with json.loads -- note how the tweets are huge JSON blobs.
Ignore tweets that error on load.
'''

class CountTweets(MRJob):
    def mapper(self, _, line):
        try:
            # Parse with: tweet = json.loads(line)
            tweet = json.loads(line)
            # do something
            if len(tweet) >= 4:
              for (key, value) in tweet:
                yield(key, 1)
        except:
            pass

    def reducer(self, key, counts):
        ### yield some output with a key of 'lines'
        #yield 'lines', some value here
        ### note that you are trying to count - so the key should remain same. So think what should be the key that is outputted from the map step. 
        yield(key, sum(counts)))

if __name__ == '__main__': 
  CountTweets.run()


## Step 1. Run MR program locally(Colab)

In [ ]:
#first run the program without catching output. Look for errors
%%timeit
!python 1_count.py -r local nashville-tweets-2019-01-28 > 1_count.out

In [ ]:
!cat 1_count.out

## Step 2. Run MR program on AWS EMR

### Please copy nashville-tweets-2019-01-28 into your S3 bucket. This is a manual process. You did this in previous assignments.

In [ ]:
#run the program. Capture output
%%timeit
!python 1_count.py -r emr s3://<s3 object url of the Tweet dataset> --cloud-tmp-dir=s3://<s3 object url of tmp directory> --cluster-id=<cluster ID> --conf-path <mrjob.conf file path in colab> > 1_count.out

In [ ]:
!cat 1_count.out

In [ ]:
#test output. Should pass if the output is correct.
f = open('1_count.out')
lines = f.readlines()
f.close()

if lines[0][:-1] in ['"lines"\t6294']:
  print("test passed")
else:
  print("test failed")